In [76]:
import pandas as pd
import math
import os.path
import time
import numpy as np
import talib 
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
#backtest lib
from backtesting import Backtest
from backtesting.lib import SignalStrategy

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [20]:
### API
binance_api_key = 'TESZdWxapwvZG90v1SJ3zMahcXySUkYv3bxRRKXKkfCMANtUS0Dw38iWALACQuHU'    #Enter your own API-key here
binance_api_secret = 'Uw1V3mbqoXDm9II7tC8lHRrYcGslHKhqo2bEXGelt08bnAn5lk4EbYfbKmHjWguY' #Enter your own API-secret here
### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "4h": 240, "1d": 1440}
batch_size = 750
# api instance
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

In [21]:
### get data funcs
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    data_df = data.astype(float)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [86]:
#get data since 2017.7.1
df = get_all_binance('BTCUSDT', '4h')

All caught up..!


In [95]:
#指標代號
critSymbol = 'MACD'
#rename to fit API
df.rename(columns={"Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"})
# calc criteria 
output = eval('talib.abstract.'+ critSymbol +'(df)')
# rename if  1-Dimention
output.name = critSymbol.lower() if type(output) == pd.core.series.Series else None
# merge result
df = pd.merge(df, pd.DataFrame(output), on = 'timestamp')
#show data
df

,macd,macdsignal,macdhist
timestamp,,,
2017-08-17 04:00:00,NaN,NaN,NaN
2017-08-17 08:00:00,NaN,NaN,NaN
2017-08-17 12:00:00,NaN,NaN,NaN
2017-08-17 16:00:00,NaN,NaN,NaN
2017-08-17 20:00:00,NaN,NaN,NaN
...,...,...,...
2020-10-30 16:00:00,85.811934,86.746465,-0.934531
2020-10-30 20:00:00,92.626341,87.922440,4.703901
2020-10-31 00:00:00,91.440520,88.626056,2.814463


In [77]:
class Strategy1(SignalStrategy):
    
    def init(self):
        super().init()
        
        # Precompute the two moving averages
        close = pd.Series(self.data.Close)
        sma1 = close.rolling(20).mean()
        sma2 = close.rolling(60).mean()
        
        # Precompute signal
        signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
        signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())
        
        signal = signal_long
        signal[signal_short] = -1
        self.set_signal(signal)
        
        
    def next(self):
        super().next()

In [125]:
backtestData = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close","volume":"Volume"})
backtestData = backtestData[["Open","High","Low","Close","Volume"]]
result1 = Backtest(backtestData, Strategy1, cash=100000).run()
print(result1)

Start                     2017-08-17 04:00:00
End                       2020-10-31 08:00:00
Duration                   1171 days 04:00:00
Exposure Time [%]                     51.6046
Equity Final [$]                       109552
Equity Peak [$]                        114285
Return [%]                            9.55209
Buy & Hold Return [%]                 216.969
Max. Drawdown [%]                    -9.09317
Avg. Drawdown [%]                   -0.547092
Max. Drawdown Duration     1049 days 00:00:00
Avg. Drawdown Duration       27 days 03:00:00
# Trades                                   68
Win Rate [%]                          30.8824
Best Trade [%]                        97.9389
Worst Trade [%]                      -16.3127
Avg. Trade [%]                        1.43795
Max. Trade Duration          35 days 04:00:00
Avg. Trade Duration           8 days 18:00:00
Profit Factor                         1.78252
Expectancy [%]                        8.72228
SQN                               